In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.datasets import make_blobs
import math

In [ ]:
def create_iris_dataset(ndims):
    iris = datasets.load_iris()
    X = iris.data[:, :ndims]
    y = iris.target
    to_take_indices = list(set(np.arange(len(y)))-set(np.where(y==2)[0]))
    X = X[to_take_indices]
    y = y[to_take_indices]
    return X, y

# 2D

In [ ]:
def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [ ]:
X, y = create_iris_dataset(ndims=2)

#...different dataset (you may comment me out):
#X, y = make_blobs(n_samples=80, centers=2, random_state=6)

In [ ]:
def plot_all(X, model, title):
    X0, X1 = X[:, 0], X[:, 1]
    xx, yy = make_meshgrid(X0, X1)
    fig, ax = plt.subplots(1,1)
    plot_contours(ax, model, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
    ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('Sepal length')
    ax.set_ylabel('Sepal width')
    ax.set_title(title)
    plt.show()
    return ax.get_xlim(), ax.get_ylim()
    

def plot_all_2(X, model, title):
    plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T
    Z = model.decision_function(xy).reshape(XX.shape)

    # plot decision boundary and margins
    ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])
    # plot support vectors
    if hasattr(model, "support_vectors_"):
        ax.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1], s=100, linewidth=1, facecolors='none', edgecolors='k')
    ax.set_title(title)
    plt.show()

In [ ]:
model = svm.LinearSVC(C=1, max_iter=10000).fit(X, y)
print("Coefficients:", model.coef_, "Intercept:", model.intercept_)
xlim, ylim = plot_all(X, model, title='LinearSVC (linear kernel)')
plot_all_2(X, model, title='LinearSVC (linear kernel)')

In [ ]:
model = svm.SVC(kernel='linear', C=1).fit(X, y)
print("Coefficients:", model.coef_, "Intercept:", model.intercept_)
xlim, ylim = plot_all(X, model, title='LinearSVC (linear kernel)')
plot_all_2(X, model, title='LinearSVC (linear kernel)')

In [ ]:
def abline(slope, intercept, xlim=(0,3), ylim=(0,3)):
    """Plot a line from slope and intercept"""
    axes = plt.gca()
    axes.set_xlim(xlim)
    axes.set_ylim(ylim)
    x_vals = np.array(axes.get_xlim())
    y_vals = -intercept + slope * x_vals
    plt.plot(x_vals, y_vals, '--')
    

def find_seperatrix(model):
    W=model.coef_[0]
    I=model.intercept_
    a = -W[0]/W[1]
    b = I[0]/W[1]
    return a, b

slope, intercept = find_seperatrix(model)

print(slope, intercept)

abline(slope, intercept, xlim, ylim)

In [ ]:
x_vals = np.array(xlim)
y_vals = -intercept + slope * x_vals
points = [np.array(i) for i in zip(x_vals, y_vals)]

vector = points[1]-points[0]

print(points)
print(vector)

In [ ]:
def show_vectors(ax, vectors, vectors_with_start):
    """https://stackoverflow.com/a/42282532/5122790"""
    HEAD_SIZE = 0.2
    M = np.array(vectors)
    colors = ['r','g','b','k','gray']
    if vectors:
        for i,l in enumerate(range(0,M.shape[0])):
            ax.arrow(0,0,M[i,0],M[i,1],head_width=HEAD_SIZE,head_length=HEAD_SIZE,color = colors[i], length_includes_head=True)

    colors = ['c','m','y']
    if vectors_with_start:
        for i,(xs,ys,xf,yf) in enumerate([(*i[0],*i[1]) for i in vectors_with_start]):
            ax.arrow(xs,ys,xf,yf,head_width=HEAD_SIZE,head_length=HEAD_SIZE,color = colors[i], length_includes_head=True)

    return ax

In [ ]:
def ortho_proj(a,b):
    #https://en.wikipedia.org/wiki/Vector_projection
    a1_sc = np.dot(a,(b/np.linalg.norm(b)))
    b_hat = b/np.linalg.norm(b)
    a1 = a1_sc*b_hat
    a2 = a-a1
    return a2

In [ ]:
center_point = points[0]+0.5*vector

In [ ]:
print(ortho_proj(vector, center_point))

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
show_vectors(ax, [], [(points[0], vector), (center_point, ortho_proj(vector, center_point))])

ax.plot(*center_point,'ok') 
ax.set_aspect('equal', 'box')
plt.grid(b=True, which='major')

plt.xlim(xlim)
plt.ylim(ylim)
plt.show()

In [ ]:
print(model.coef_[0])
print(center_point+model.coef_[0])

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
show_vectors(ax, [], [(points[0], vector), (center_point, ortho_proj(vector, center_point)), (center_point, model.coef_[0])])

ax.plot(*center_point,'ok') 
ax.set_aspect('equal', 'box')
plt.grid(b=True, which='major')

plt.xlim(xlim)
plt.ylim(ylim)
plt.show()

# 3D

In [ ]:
#https://stackoverflow.com/a/51301399/5122790 !!!
import plotly.express as px
import plotly.graph_objects as go
from mpl_toolkits import mplot3d

In [ ]:
X, y = create_iris_dataset(ndims=3)
model = svm.LinearSVC(C=1, max_iter=10000).fit(X, y)
print("Coefficients:", model.coef_, "Intercept:", model.intercept_)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = plt.axes(projection='3d')
ax.scatter(X[:,0], X[:,1], X[:,2], color=["r" if i==0 else "b" for i in y])
plt.show()

In [ ]:
def create_3d_meshgrid_xy(X, amount=30):
    lsx = np.linspace(min(X[:,0])-0.5, max(X[:,0])-0.5, amount)
    lsy = np.linspace(min(X[:,1])-0.5, max(X[:,1])-0.5, amount)
    xx, yy = np.meshgrid(lsx,lsy)
    return xx, yy


def plot_3d_boundary(clf):
    # The equation of the separating plane is given by all x so that np.dot(svc.coef_[0], x) + b = 0.
    # Solve for w3 (z)
    z = lambda clf, x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]
    # https://stackoverflow.com/a/51301399/5122790 !!!

    xx, yy = create_3d_meshgrid_xy(X)

    fig = plt.figure()
    ax  = fig.add_subplot(111, projection='3d')
    ax.plot3D(X[y==0,0], X[y==0,1], X[y==0,2],'ob')
    ax.plot3D(X[y==1,0], X[y==1,1], X[y==1,2],'sr')
    ax.plot_surface(xx, yy, z(clf,xx,yy))
    ax.view_init(30, 60)
    plt.show()
    
plot_3d_boundary(model)

In [ ]:
def ortho_projection_affine(a, b):
    """https://en.wikipedia.org/wiki/Vector_projection"""
    return np.dot(np.dot(a,b)/np.dot(b,b),b)

def create_3d_figure():
    #https://community.plotly.com/t/creating-a-3d-scatterplot-with-equal-scale-along-all-axes/15108/7
    return go.Figure(layout=go.Layout(
                        scene=dict(camera=dict(eye=dict(x=1, y=1, z=1)), aspectmode="data"),
                        autosize=True,
                        width=1000,
                        height=800,
                        margin=dict(l=10, r=10, b=10, t=10, pad=4),
                        paper_bgcolor="White"))

def add_surface(fig, x, y, z):
    fig.add_trace(go.Surface(x=x, y=y, z=z))


def add_markers(fig, points, color="black", size=2):
    points = np.array(points)
    if points.ndim == 1: points = np.array([points])
    fig.add_trace(
        go.Scatter3d(
            mode='markers',
            x=points[:,0],
            y=points[:,1],
            z=points[:,2],
            marker={"color": color,
                    "size": size,
                    "line": {"width": 0}
                   },
        )
    )
    
def add_line(fig, point1, point2, width=6):
    #https://stackoverflow.com/questions/43164909/plotlypython-how-to-plot-arrows-in-3d
    fig.add_trace(
        go.Scatter3d(x = [point1[0], point2[0]],
                     y = [point1[1], point2[1]],
                     z = [point1[2], point2[2]],
                     marker = dict(size = 1),
                     line = dict(width = width)
                     )
    )
    
def add_sample_projections(fig, X, onto, n_samples=10):
    show_vecs = X[np.random.choice(X.shape[0], n_samples, replace=False), :]
    for point in show_vecs:
        proj = ortho_projection_affine(point, onto)
        add_line(fig, point, proj)

In [ ]:
def plotly_complete_3d_plot(model, X, y):
    z = lambda clf, x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]

    fig = create_3d_figure()
    add_markers(fig, X, color=y) #samples

    xx, yy = create_3d_meshgrid_xy(X)                
    add_surface(fig, xx, yy, z(model,xx,yy)) #decision hyperplane

    add_line(fig, X.mean(axis=0)-model.coef_[0], X.mean(axis=0)+model.coef_[0]) #orthogonal of decision hyperplane through mean of points
    add_line(fig, -model.coef_[0]*5, model.coef_[0]*5) #orthogonal of decision hyperplane through [0,0,0]
    add_markers(fig, [0,0,0], size=3) #coordinate center
    add_sample_projections(fig, X, model.coef_[0]) #orthogonal lines from the samples onto the decision hyperplane orthogonal

    return fig

plotly_complete_3d_plot(model, X, y)

In [ ]:
X, y = make_blobs(n_samples=80, centers=2, random_state=6, n_features=3)
model = svm.LinearSVC(C=1, max_iter=10000).fit(X, y)
print("Coefficients:", model.coef_, "Intercept:", model.intercept_)
plotly_complete_3d_plot(model, X, y)

* I'm having so many apparently hard-to-solve problems - the decision-boundary for example goes from (minx,miny) to (maxx,maxy), with the corresponding z-coordinate. However that's superflous, it may become waaay to big there, because the original x/y/z coordinate system is irrelevant! The size of the decision surface is constrained instead rather by the circle of all lines that go through a point and in parallel to the orthogonal of the decision surface. But to figure out the size of the decision boundary from that is far from trivial - again because I always need the translation from the irrelevant acutal xyz-coordinate system to the coordinate system which is relevant for me, which is one in which the decision surface has two coordinate values equal to zero. If I do that, every other calculation becomes trivial!!
* Same holds for the problem "how long should the decisionsurface-orthogonal be" - calculating it now is hard, but once we have translated the coordinate system it's just the min/max z-coordinate!

In [ ]:
def create_3d_meshgrid_xy(X, amount=30):
    lsx = np.linspace(min(X[:,0])-0.5, max(X[:,0])-0.5, amount)
    lsy = np.linspace(min(X[:,1])-0.5, max(X[:,1])-0.5, amount)
    xx, yy = np.meshgrid(lsx,lsy)
    return xx, yy


def add_surface(fig, x, y, z):
    fig.add_trace(go.Surface(x=x, y=y, z=z))

    
X, y = make_blobs(n_samples=80, centers=2, random_state=6, n_features=3)
model = svm.LinearSVC(C=1, max_iter=10000).fit(X, y)
print("Coefficients:", model.coef_, "Intercept:", model.intercept_)



z = lambda clf, x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]

fig = create_3d_figure()
add_markers(fig, X, color=y) #samples

xx, yy = create_3d_meshgrid_xy(X)                
add_surface(fig, xx, yy, z(model,xx,yy)) #decision hyperplane

add_line(fig, X.mean(axis=0)-model.coef_[0], X.mean(axis=0)+model.coef_[0]) #orthogonal of decision hyperplane through mean of points
add_line(fig, [0,0,0], model.coef_[0]*100) #orthogonal of decision hyperplane through [0,0,0]
add_markers(fig, [0,0,0], size=3) #coordinate center
#add_sample_projections(fig, X, model.coef_[0]) #orthogonal lines from the samples onto the decision hyperplane orthogonal

for point, side in zip(X, y):
    add_line(fig, point, point-model.coef_[0]*100 if side else point+model.coef_[0]*100)

fig

In [ ]:
vec = model.coef_[0]

normalize = lambda vec: vec/np.linalg.norm(vec)
print("Original Vec:", normalize(vec))

fig = create_3d_figure()
add_line(fig, [0,0,0], normalize(vec), width=12)
add_line(fig, [-1,0,0], [1,0,0])
add_line(fig, [0,-1,0], [0,1,0])
add_line(fig, [0,0,-1], [0,0,1])

add_line(fig, [0,0,0], normalize([0, vec[1], vec[2]]))
add_line(fig, [0,0,0], normalize([vec[0], 0, vec[2]]))
add_line(fig, [0,0,0], normalize([vec[0], vec[1], 0]))

print()
print(normalize([0, vec[1], vec[2]]))
print(normalize([vec[0], 0, vec[2]]))
print(normalize([vec[0], vec[1], 0]))


fig

In [ ]:
vec = model.coef_[0]
angle = np.arctan2([1, 0, 0], normalize(vec))
angle = angle.squeeze()[0] #die anderen beiden sind pi (logisch, weil ich ja den vector nehme wo y=z=0?)
print("Angle:", angle)
rotation_matrix = lambda theta: np.array([[np.cos(theta), np.sin(theta),0],[-np.sin(theta), np.cos(theta),0],[0,0,1]])
mat = rotation_matrix(angle)
print("Rotation-Matrix:\n", mat)
print()

print(mat.dot(vec))
print(mat.dot(np.array([1,0,0])))

In [ ]:
#...ok, so nicht. Lass uns erstmal 3 Unit vectors finden.
uvec1 = model.coef_[0]
z = lambda clf, x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]

#wir haben im 3D einen Degree of freedom hier!! uvec1 ist eindeutig definiert, aber uvec2 und uvec3 haben nur
#die Bedingung dass sie orthogonal zueinander und zu uvec1 sind - das ist 1 dof. Im 2D haben wir 0 dof, da die 
#decisionboundary 1dimensional ist und ihre orthogonale auch. Wie viele dof haben wir im höherdim? immer n_dim-2? 
#das hat ja mega auswirkungen auf was ich machen will, am besten wäre es die neuen basen so zu wählen dass die
#jeweils nächsten auch wieder orthogonal sind...!

uvec2 = np.array([1, 0, z(model, 1, 0)])
uvec3 = np.cross(uvec1, uvec2)

uvec1, uvec2, uvec3 = normalize(uvec1), normalize(uvec2), normalize(uvec3)
fig = create_3d_figure()
add_line(fig, [0,0,0], uvec1)
add_line(fig, [0,0,0], uvec2)
add_line(fig, [0,0,0], uvec3)

back_trafo_matrix = np.array([uvec2, uvec3, uvec1]).T
#https://youtu.be/P2LTAUO1TdA?t=340: a matrix whose columns represents J's Basis Vectors is a transformation that 
#moves our basis vectors to J's.
trafo_matrix = np.linalg.inv(back_trafo_matrix)

fig

In [ ]:
def ortho_projection_affine(a, b):
    """https://en.wikipedia.org/wiki/Vector_projection"""
    return np.dot(np.dot(a,b)/np.dot(b,b),b)


def generate_trafo_matrices(model, z_func):
    normalize = lambda vec: vec/np.linalg.norm(vec)
    uvec1 = normalize(model.coef_[0])
    uvec2 = normalize(np.array([1, 0, z_func(model, 1, 0)]))
    uvec3 = normalize(np.cross(uvec1, uvec2))
    back_trafo_matrix = np.array([uvec2, uvec3, uvec1]).T
    trafo_matrix = np.linalg.inv(back_trafo_matrix)
    return trafo_matrix, back_trafo_matrix


class ThreeDFigure():
    def __init__(self, trafo_mat=None, intercept=0):
        self.trafo_mat = trafo_mat if trafo_mat is not None else np.eye(3)
        self.inverse_trafo_mat = np.linalg.inv(self.trafo_mat)
        self.intercept = intercept
        #https://community.plotly.com/t/creating-a-3d-scatterplot-with-equal-scale-along-all-axes/15108/7
        self.fig = go.Figure(layout=go.Layout(
                        scene=dict(camera=dict(eye=dict(x=1, y=1, z=1)), aspectmode="data"),
                        autosize=True,
                        width=1000,
                        height=800,
                        margin=dict(l=10, r=10, b=10, t=10, pad=4),
                        paper_bgcolor="White"))
        
    def _transform(self, points, inverse=False):
        points = np.array(points)
        if points.ndim == 1: points = np.array([points])
        trafo_mat = self.inverse_trafo_mat if inverse else self.trafo_mat      
        intercept = self.intercept if inverse else -self.intercept
        return np.array([trafo_mat.dot(point)+np.array([0,0,intercept]) for point in points])
                    
    def add_surface(self, x, y, z):
        x_shape = x.shape
        points = np.stack([x, y, z],axis=2).reshape(-1,3)
        points = self._transform(points)
        points = points.reshape(*x_shape, 3)
        self.fig.add_trace(go.Surface(x=points[:,:,0], y=points[:,:,1], z=points[:,:,2]))
    
    def add_markers(self, points, color="black", size=2):
        points = np.array(points)
        if points.ndim == 1: points = np.array([points])
        points = self._transform(points)
        self.fig.add_trace(
            go.Scatter3d(
                mode='markers',
                x=points[:,0],
                y=points[:,1],
                z=points[:,2],
                marker={"color": color,
                        "size": size,
                        "line": {"width": 0}
                       },
            )
        )
    
    def add_line(self, point1, point2, width=6):
        #https://stackoverflow.com/questions/43164909/plotlypython-how-to-plot-arrows-in-3d
        point1, point2 = self._transform(point1).squeeze(), self._transform(point2).squeeze()
        self.fig.add_trace(
            go.Scatter3d(x = [point1[0], point2[0]],
                         y = [point1[1], point2[1]],
                         z = [point1[2], point2[2]],
                         marker = dict(size = 1),
                         line = dict(width = width)
                         )
        )
    
    def add_sample_projections(self, X, onto, n_samples=10):
        #no need to transform here, self.add_line already does transformation!
        show_vecs = X[np.random.choice(X.shape[0], n_samples, replace=False), :]
        for point in show_vecs:
            proj = ortho_projection_affine(point, onto)
            self.add_line(point, proj)
            
    def show(self):
        return self.fig

In [ ]:
X, y = make_blobs(n_samples=80, centers=2, random_state=6, n_features=3)
model = svm.LinearSVC(C=1, max_iter=10000).fit(X, y)
z = lambda clf, x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]
trafo_matrix, back_trafo_matrix = generate_trafo_matrices(model, z)

print("Coefficients:", model.coef_, "Intercept:", model.intercept_)


fig = ThreeDFigure(trafo_mat=trafo_matrix)#, intercept=model.intercept_[0])

fig.add_markers(X, color=y) #samples

xx, yy = create_3d_meshgrid_xy(X)                
fig.add_surface(xx, yy, z(model,xx,yy)) #decision hyperplane

fig.add_line(X.mean(axis=0)-model.coef_[0], X.mean(axis=0)+model.coef_[0]) #orthogonal of decision hyperplane through mean of points
fig.add_line([0,0,0], model.coef_[0]*100) #orthogonal of decision hyperplane through [0,0,0]
fig.add_markers([0,0,0], size=3) #coordinate center
fig.add_sample_projections(X, model.coef_[0]) #orthogonal lines from the samples onto the decision hyperplane orthogonal

for point, side in zip(X, y):
    fig.add_line(point, point-model.coef_[0]*100 if side else point+model.coef_[0]*100)

fig.show()

In [ ]:
X, y = make_blobs(n_samples=80, centers=2, random_state=6, n_features=3)
model = svm.LinearSVC(C=1, max_iter=10000).fit(X, y)
z = lambda clf, x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]
trafo_matrix, back_trafo_matrix = generate_trafo_matrices(model, z)


fig = ThreeDFigure(trafo_mat=trafo_matrix)

fig.add_markers(X, color=y) #samples

xx, yy = create_3d_meshgrid_xy(X)                
fig.add_surface(xx, yy, z(model,xx,yy)) #decision hyperplane

fig.add_line(X.mean(axis=0)-model.coef_[0], X.mean(axis=0)+model.coef_[0]) #orthogonal of decision hyperplane through mean of points
fig.add_line([0,0,0], model.coef_[0]*100) #orthogonal of decision hyperplane through [0,0,0]
fig.add_markers([0,0,0], size=3) #coordinate center
fig.add_sample_projections(X, model.coef_[0]) #orthogonal lines from the samples onto the decision hyperplane orthogonal

for point, side in zip(X, y):
    transformed = fig._transform(point)[0]
    fig.add_line(point, fig._transform([transformed[0], transformed[1], 0], inverse=True))

fig.show()

In [ ]:
#trying manually... without intercept: 

vec = np.array([0.1, -0.4, -0.5])
plane_func = lambda vec, x,y: (-vec[0]*x -vec[1]*y) / vec[2]

def make_meshgrid(val=0.1, amount=30):
    lsx = np.linspace(-val, val, amount)
    lsy = np.linspace(-val, val, amount)
    xx, yy = np.meshgrid(lsx,lsy)
    return xx, yy

def generate_trafo_matrices(uvec1, z_func):
    normalize = lambda vec: vec/np.linalg.norm(vec)
    uvec2 = normalize(np.array([1, 0, z_func(vec, 1, 0)]))
    uvec3 = normalize(np.cross(uvec1, uvec2))
    back_trafo_matrix = np.array([uvec2, uvec3, uvec1]).T
    trafo_matrix = np.linalg.inv(back_trafo_matrix)
    return trafo_matrix, back_trafo_matrix

trafo_matrix, back_trafo_matrix = generate_trafo_matrices(vec, plane_func)

xx, yy = make_meshgrid(0.2)

fig = ThreeDFigure(trafo_matrix)
fig.add_surface(xx, yy, plane_func(vec,xx,yy))
fig.add_line([0,0,0],vec)
fig.show()

Problem: wir machen zwar basiswechsel, gehen aber davon aus dass der Ursprung an der gleichen Position ist Ebene hat den Intercept 0, geht also durch [0,0,0]. Das ist aber nicht so! Und in unserer Transformationsmatrix können wir das nicht ausdrücken, da trafomatrizen zum basiswechsel von affinen Vektrorräumen ausgehen, bei denen der urpsrung an der selben position ist. DAS HEIßT ZUSÄTZLICH zum basiswechsel mit matrizenmultiplikation müssen wir den punkt des neuen ursprugns draufrechnen. Und dieser Punkt ist IM ALTEN KOORDINATENSYSTEM intercept*orthongonalvektor.

In [ ]:
#trying manually... with intercept: 

class ThreeDFigure():
    def __init__(self, trafo_mat=None, intercept=0):
        self.trafo_mat = trafo_mat if trafo_mat is not None else np.eye(3)
        self.inverse_trafo_mat = np.linalg.inv(self.trafo_mat)
        self.intercept = intercept
        #https://community.plotly.com/t/creating-a-3d-scatterplot-with-equal-scale-along-all-axes/15108/7
        self.fig = go.Figure(layout=go.Layout(
                        scene=dict(camera=dict(eye=dict(x=1, y=1, z=1)), aspectmode="data"),
                        autosize=True,
                        width=1000,
                        height=800,
                        margin=dict(l=10, r=10, b=10, t=10, pad=4),
                        paper_bgcolor="White"))
        
    def _transform(self, points, inverse=False):
        points = np.array(points)
        if points.ndim == 1: points = np.array([points])
        trafo_mat = self.inverse_trafo_mat if inverse else self.trafo_mat      
        intercept = self.intercept if inverse else -self.intercept
        #ursprung_translate = trafo_mat.dot(np.array([0,0,0])+trafo_matrix[:,0]*intercept)
        
        #points = np.array([point+trafo_matrix[:,0]*intercept for point in points])
        points = np.array([trafo_mat.dot(point) for point in points])        
        #points = np.array([point-ursprung_translate for point in points])
        return points
                    
    def add_surface(self, x, y, z_func):
        xy_arr = np.vstack([xx.flatten(), yy.flatten()]).T
        z_arr = np.array([z_func(*i) for i in xy_arr])
        points = np.column_stack([xy_arr, z_arr])
        tmp = points[:,2]
        points = self._transform(points)
        surface_form = lambda x: x.reshape(round(math.sqrt(x.shape[0])),-1)
        self.fig.add_trace(go.Surface(x=surface_form(points[:,0]), y=surface_form(points[:,1]), z=surface_form(points[:,2])))
    
    def add_line(self, point1, point2, width=6):
        #https://stackoverflow.com/questions/43164909/plotlypython-how-to-plot-arrows-in-3d
        point1, point2 = self._transform(point1).squeeze(), self._transform(point2).squeeze()
        self.fig.add_trace(
            go.Scatter3d(x = [point1[0], point2[0]],
                         y = [point1[1], point2[1]],
                         z = [point1[2], point2[2]],
                         marker = dict(size = 1),
                         line = dict(width = width)
                         )
        )
        
    def show(self):
        return self.fig

def make_meshgrid(val=0.1, amount=30):
    lsx = np.linspace(-val, val, amount)
    lsy = np.linspace(-val, val, amount)
    xx, yy = np.meshgrid(lsx,lsy)
    return xx, yy

def generate_trafo_matrices(uvec1, z_func, intercept):
    normalize = lambda vec: vec/np.linalg.norm(vec)
    uvec2 = normalize(np.array([1, 0, z_func(uvec1, 1, 0)]))
    uvec3 = normalize(np.cross(uvec1, uvec2))
    back_trafo_matrix = np.array([uvec2, uvec3, uvec1]).T
    trafo_matrix = np.linalg.inv(back_trafo_matrix)
    return trafo_matrix, back_trafo_matrix


###################################

vec = np.array([-0.14, -0.02, -0.05])
interc = -0.005
plane_func = lambda vec, x,y: (-interc-vec[0]*x-vec[1]*y) / vec[2]

trafo_matrix, back_trafo_matrix = generate_trafo_matrices(vec, plane_func, interc)

xx, yy = make_meshgrid(0.2)

fig = ThreeDFigure(trafo_matrix, interc)
fig.add_surface(xx, yy, lambda xx, yy: plane_func(vec,xx,yy))
fig.add_line([0,0,0],vec)

fig.add_line([0,0,0], fig._transform([0.1,0,0], inverse=True))
fig.add_line([0,0,0], fig._transform([0,0.1,0], inverse=True))
fig.add_line([0,0,0], fig._transform([0,0,0.1], inverse=True))

print(fig._transform([0,0,0], inverse=True))

fig.show()

In [ ]:
1